# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [20]:
from azureml.core import Workspace, Experiment

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
from azureml.core.experiment import Experiment
import os

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [21]:
ws = Workspace.from_config()
experiment_name = 'hyperparameter-tuning-yum'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-223541
Azure region: southcentralus
Subscription id: 9e65f93e-bdd8-437b-b1e8-0647cd6098f7
Resource group: aml-quickstarts-223541


In [23]:
# validate dataset is in store
key = "maternal-health"
description_text = "Maternal health"


if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print('The Dataset was found')
# else:
#     # Create AML Dataset and register it into Workspace
#     data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00639/Maternal Health Risk Data Set.csv"
#     dataset = Dataset.Tabular.from_delimited_files(data_url)
#     #Register Dataset in Workspace
#     dataset = dataset.register(workspace = ws,name = key,description = description_text)

df = dataset.to_pandas_dataframe()
df.head()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,98.0,86,high risk
1,35,140,90,13.0,98.0,70,high risk
2,29,90,70,8.0,100.0,80,high risk
3,30,140,85,7.0,98.0,70,high risk
4,35,120,60,6.1,98.0,76,low risk


## Create or Attach an AmlCompute cluster

In [24]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cluster-yum"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [25]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1,evaluation_interval=1, delay_evaluation=5 )

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "C": choice(0.1, 1, 5),
    "max_iter": choice(25, 50, 100, 200, 500)
}
)
#TODO: Create your estimator and hyperdrive config
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory=".", script="train.py", compute_target=compute_target, environment=sklearn_env)
# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src, hyperparameter_sampling=param_sampling, policy=early_termination_policy, primary_metric_name="Accuracy", primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=4, max_concurrent_runs=4)

In [31]:
#TODO: Submit your experiment
from azureml.core.experiment import Experiment
run = experiment.submit(config=hyperdrive_config)
run.wait_for_completion(show_output=True)

RunId: HD_93dea33b-89c4-453a-ab75-105cb64905b7
Web View: https://ml.azure.com/runs/HD_93dea33b-89c4-453a-ab75-105cb64905b7?wsid=/subscriptions/9e65f93e-bdd8-437b-b1e8-0647cd6098f7/resourcegroups/aml-quickstarts-223541/workspaces/quick-starts-ws-223541&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-01-26T08:10:47.455682][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-01-26T08:10:48.3437458Z][SCHEDULER][INFO]Scheduling job, id='HD_93dea33b-89c4-453a-ab75-105cb64905b7_0' 
[2023-01-26T08:10:48.4477131Z][SCHEDULER][INFO]Scheduling job, id='HD_93dea33b-89c4-453a-ab75-105cb64905b7_1' 
[2023-01-26T08:10:48.5669935Z][SCHEDULER][INFO]Scheduling job, id='HD_93dea33b-89c4-453a-ab75-105cb64905b7_2' 
[2023-01-26T08:10:48.6691945Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_93dea33b-89c4-453a-ab75-105cb64905b7_0' 
[2023-01-26T08:10:48.6996306Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_93dea33b-89c4-453a-ab75-

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Execution failed. User process '/azureml-envs/azureml_7eec2c8971b9410f92147a7e257297e7/bin/python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error details. Error:   File \"train.py\", line 48, in <module>\n    main()\n  File \"train.py\", line 40, in main\n    model = LogisticRegression(C=args.C, max_iter=args.max_iter).fit(x_train, y_train)\n  File \"/azureml-envs/azureml_7eec2c8971b9410f92147a7e257297e7/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py\", line 1346, in fit\n    accept_large_sparse=solver != 'liblinear')\n  File \"/azureml-envs/azureml_7eec2c8971b9410f92147a7e257297e7/lib/python3.6/site-packages/sklearn/base.py\", line 433, in _validate_data\n    X, y = check_X_y(X, y, **check_params)\n  File \"/azureml-envs/azureml_7eec2c8971b9410f92147a7e257297e7/lib/python3.6/site-packages/sklea

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [28]:
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

KeyError: 'log_files'

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [29]:
import joblib
# Get your best run and save the model from that run.
best_run = run.get_best_run_by_primary_metric()
best_run.get_details()

{'runId': 'HD_0ba5450b-b1ca-47ab-a60b-e77e687cc89a_1',
 'target': 'cluster-yum',
 'status': 'Completed',
 'startTimeUtc': '2023-01-26T07:48:51.076879Z',
 'endTimeUtc': '2023-01-26T07:49:15.458843Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '40cff598-99b6-494c-87f6-d9461f4851fc',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--C', '1', '--max_iter', '200'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'cluster-yum',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'e

In [10]:
best_run_metrics = best_run.get_metrics()
print('Accuracy:', best_run_metrics["Accuracy"])

Accuracy: 0.9452054794520548


In [11]:
#TODO: Save the best model
best_run.download_file("model.pkl")

In [12]:
model = best_run.register_model(model_name="model.pkl")
print(model.name, model.id, model.version, sep='\t')

model.pkl	model.pkl:1	1


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

